**Map Armed Conflict Location in Afghanistan and Pakistan with Folum.** 

This notebook loads daily data from The Armed Conflict Location & Event Data Project (ACLED); https://www.acleddata.com/ and plot the data on a map. The markers size and color correlated to number of casualties. 


In [1]:
import pandas as pd
import folium
import folium.plugins
import branca.colormap as cm

Read data on armed conflicts in the last month from https://www.acleddata.com/data/ into a dataframe

See API here: https://www.acleddata.com/wp-content/uploads/dlm_uploads/2017/10/API-User-Guide-11.pdf

You can slao load the data from "2019-01-01-2019-01-31-Afghanistan-Pakistan.csv" file in this directory

In [2]:
def load_data():

    start = (pd.datetime.now()-pd.Timedelta(1, unit='M')).strftime("%Y-%m-%d")
    end = pd.datetime.now().strftime("%Y-%m-%d")
    q = "?limit=0&country=Afghanistan|Pakistan&event_date={0}|{1}&event_date_where=BETWEEN".format(start,end)
    event_date = ["event_date"]
    # First send get / read command with terms=accept to accept ACLEDDATA terms and conditions
    df = pd.read_csv("https://api.acleddata.com/acled/read?terms=accept")
    try:
        df = pd.read_csv("https://api.acleddata.com/acled/read.csv{}&terms=accept".format(q),
                         parse_dates=event_date, date_parser=pd.to_datetime)
        # Make sure the API returned data
        assert(df.shape[1] > 2)
    except AssertionError as e:
        print("Error loading data from API {0}. Loading data from local file".format(e))
        df = pd.read_csv("2019-01-01-2019-01-31-Afghanistan-Pakistan.csv", 
                         parse_dates=event_date, date_parser=pd.to_datetime)
    return df

Generate color map ranging from yellow to red and depends of number of fatalities

In [3]:
def get_color_map(df):
    
    cmap = cm.linear.YlOrRd_09.scale(df.fatalities.min(), vmax=df.fatalities.max())
    cmap.caption = "Number of fatalities"
    return cmap

Help fuction to create geojason from the data to feed into the TimestampedGeoJson plugin of Folium. This plugin requires geojson input features.

In [4]:
def to_geo_jason(df, cmap):
    
    features = []
    for _, row in df.iterrows():
        popup = "location: {0}, {1}, actors: {2}-{3}, Comments: {4}".format(row.location, row.country, row.actor1, row.actor2, row.notes)
        feature = {
            "type": "Feature",
            "geometry": {
                        "type":"Point",
                        "coordinates":[row.longitude,row.latitude]
                        },
            "properties": {
                            "time": (row.event_date+pd.Timedelta(1, unit='d')).strftime("%Y-%m-%d"),
                            "style": {"color" : "black"
                                     },
                            "icon": "circle",
                            "iconstyle":{
                                "fillColor": cmap(row.fatalities),
                                "fillOpacity": 0.8,
                                "stroke": "true",
                                "radius": int(row.fatalities+4)/2},
                            "popup": popup
                         },
                 }
        features.append(feature)
    return features

Create a foliun map and add the geojason features to it. Use foliu's TimestampedGeoJson plugin has neat options for the time slider. (see https://python-visualization.github.io/folium/plugins.html)

In [5]:
def build_map(features, cmap):
    attacks_map = folium.Map(location=[34,63], zoom_start=5)
    
    folium.plugins.TimestampedGeoJson(        
                        {'type': 'FeatureCollection',
                         'features': features}
                         , period='P1D'
                         , add_last_point=True
                         , auto_play=False
                         , loop=False
                         , max_speed=1
                         , loop_button=True
                         , date_options='YYYY/MM/DD'
                         , time_slider_drag_update=True
                         , duration = 'PT23H' 
    ).add_to(attacks_map)
    attacks_map.add_child(cmap)
    folium.plugins.MousePosition(prefix="Mouse:").add_to(attacks_map)
    return attacks_map

In [6]:
df = load_data()
cmap = get_color_map(df)
geo_jason_data = to_geo_jason(df, cmap)
attacks_map = build_map(geo_jason_data, cmap)
attacks_map

In [7]:
attacks_map.save("armed_conflicts_afg_pak.html")